In [3]:
!pip install pyathena

  Using cached pyathena-3.9.0-py3-none-any.whl.metadata (6.3 kB)
Using cached pyathena-3.9.0-py3-none-any.whl (75 kB)


In [1]:
from sagemaker import get_execution_role
from time import gmtime, strftime, sleep
from sagemaker.feature_store.feature_group import FeatureGroup
import pyathena
from pyathena import connect
import boto3
import pandas as pd
import numpy as np
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.feature_store.feature_definition import FeatureDefinition, FeatureTypeEnum
import io
import time

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [2]:
# Set up session
region = boto3.Session().region_name
sagemaker_session = Session(boto3.Session(region_name=region))
s3_client = boto3.client("s3", region_name=region)
role = get_execution_role()


In [3]:
# S3 bucket and data paths
bucket_name = "housingdatafiles"
housing_key = "housing.csv"
gmaps_key = "housing_gmaps_data_raw.csv"
merged_file_key = "housing3.csv"

In [5]:
# Load data
housing_data = pd.read_csv(io.BytesIO(s3_client.get_object(Bucket=bucket_name, Key=housing_key)['Body'].read()))
gmaps_data = pd.read_csv(io.BytesIO(s3_client.get_object(Bucket=bucket_name, Key=gmaps_key)['Body'].read()))

# Forward fill missing neighborhood data
gmaps_data['neighborhood-political'].ffill(inplace=True)

# Merge datasets on latitude and longitude
housing3 = housing_data.merge(gmaps_data[['latitude', 'longitude', 'neighborhood-political']], on=['latitude', 'longitude'], how='left')

# Define "neighborhood" column
housing3['neighborhood'] = housing3['neighborhood-political']

/tmp/ipykernel_186/1349734240.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gmaps_data['neighborhood-political'].ffill(inplace=True)


In [6]:
database_name = "housingd_data3" # create database name

In [7]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket_name)

In [8]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir) # create connection

In [9]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name) # Testing the database
print(statement)

CREATE DATABASE IF NOT EXISTS housingd_data3


In [10]:
pd.read_sql(statement, conn)

/tmp/ipykernel_186/618921652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [21]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show

/tmp/ipykernel_22/2126029005.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,healthcare_analytics
2,housingd_data3
3,sagemaker_featurestore


In [11]:
from sagemaker.session import Session

region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)


In [12]:
# call for the role information.
role = get_execution_role()
print(role)

arn:aws:iam::002969758287:role/LabRole


In [ ]:
merged_file_key = 'housing3.csv'

In [13]:
# Set Athena parameters
database_name = "housingd_data3"
table_name_csv = merged_file_key

In [15]:
default_s3_bucket_name = feature_store_session.default_bucket()
prefix = "housing_data3/data/feature-store"
s3_uri=f"s3://{bucket_name}/{prefix}"

s3_uri

's3://housingdatafiles/housing_data3/data/feature-store'

In [16]:
baseline_feature_group = FeatureGroup(
    name='baseline_feature_group', sagemaker_session=feature_store_session
)

In [22]:
import time
import pandas as pd

# Load your CSV file into a DataFrame
merged_file_path = "s3://housingdatafiles/housing3.csv"  
merged_file_key = pd.read_csv(merged_file_path)  # Now merged_file_key is a DataFrame


# Get the current time in seconds for the event_time feature
current_time_sec = int(round(time.time()))

# Define helper function to cast object types to string
def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        if data_frame.dtypes[label] == "object":
            data_frame[label] = data_frame[label].astype("str").astype("string")

# Cast object dtype to string
cast_object_to_string(merged_file_key)

# Convert boolean columns to integers for compatibility with Feature Store
for label in merged_file_key.select_dtypes(include=['bool']).columns:
    merged_file_key[label] = merged_file_key[label].astype("int")

# Define record identifier and event time feature names
record_identifier_feature_name = "neighborhood"
event_time_feature_name = "event_time"

# Append event_time feature to the DataFrame
merged_file_key[event_time_feature_name] = pd.Series(
    [current_time_sec] * len(merged_file_key), dtype="float64" 
)

# Load feature definitions into the feature group
# Replace baseline_feature_group with the actual feature group instance
baseline_feature_group.load_feature_definitions(data_frame=merged_file_key)



[FeatureDefinition(feature_name='longitude', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='latitude', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='housing_median_age', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='total_rooms', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='total_bedrooms', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='population', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='households', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='median_income', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, co

In [26]:
# Delete the existing feature group by name
baseline_feature_group.delete()
print("Existing feature group deleted. Recreating with the same name.")

Existing feature group deleted. Recreating with the same name.


In [27]:
import time

# Standardize feature names to remove unsupported characters
merged_file_key.columns = merged_file_key.columns.str.replace("<1H OCEAN", "1h_ocean", regex=False)
merged_file_key.columns = merged_file_key.columns.str.replace("NEAR BAY", "near_bay", regex=False)
merged_file_key.columns = merged_file_key.columns.str.replace("NEAR OCEAN", "near_ocean", regex=False)
merged_file_key.columns = merged_file_key.columns.str.replace(" ", "_", regex=False)  # For any other spaces

# Verify column names are cleaned up
print("Updated column names:", merged_file_key.columns)

# Add event time feature
current_time_sec = int(round(time.time()))
merged_file_key['event_time'] = pd.Series([current_time_sec] * len(merged_file_key), dtype="float64")

# Define feature definitions for the Feature Group
features = [
    FeatureDefinition(feature_name=col, feature_type=FeatureTypeEnum.FRACTIONAL if merged_file_key[col].dtype == 'float64' else FeatureTypeEnum.INTEGRAL)
    if col != "neighborhood" else FeatureDefinition(feature_name="neighborhood", feature_type=FeatureTypeEnum.STRING)
    for col in merged_file_key.columns
]

# Load feature definitions to the feature group
baseline_feature_group.load_feature_definitions(data_frame=merged_file_key)

# Re-create the Feature Group
try:
    baseline_feature_group.create(
        s3_uri="s3://housingdatafiles/feature-store4",
        record_identifier_name="neighborhood",
        event_time_feature_name="event_time",
        role_arn=role,
        enable_online_store=True,
    )
    print("Feature Group creation initiated.")
    wait_for_feature_group_creation_complete(baseline_feature_group)
except ClientError as e:
    print("Error creating feature group:", e)


Updated column names: Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity', 'neighborhood-political',
       'neighborhood', 'ocean_proximity_1h_ocean', 'ocean_proximity_INLAND',
       'ocean_proximity_ISLAND', 'ocean_proximity_near_bay',
       'ocean_proximity_near_ocean', 'median_house_age_grouped',
       'bedrooms_per_household', 'event_time'],
      dtype='object')
Feature Group creation initiated.
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup baseline_feature_group successfully created.


In [28]:
baseline_feature_group.ingest(data_frame=merged_file_key, max_workers=5, wait=True)

IngestionManagerPandas(feature_group_name='baseline_feature_group', feature_definitions={'longitude': {'FeatureName': 'longitude', 'FeatureType': 'Fractional'}, 'latitude': {'FeatureName': 'latitude', 'FeatureType': 'Fractional'}, 'housing_median_age': {'FeatureName': 'housing_median_age', 'FeatureType': 'Fractional'}, 'total_rooms': {'FeatureName': 'total_rooms', 'FeatureType': 'Fractional'}, 'total_bedrooms': {'FeatureName': 'total_bedrooms', 'FeatureType': 'Fractional'}, 'population': {'FeatureName': 'population', 'FeatureType': 'Fractional'}, 'households': {'FeatureName': 'households', 'FeatureType': 'Fractional'}, 'median_income': {'FeatureName': 'median_income', 'FeatureType': 'Fractional'}, 'median_house_value': {'FeatureName': 'median_house_value', 'FeatureType': 'Fractional'}, 'ocean_proximity': {'FeatureName': 'ocean_proximity', 'FeatureType': 'String'}, 'neighborhood-political': {'FeatureName': 'neighborhood-political', 'FeatureType': 'String'}, 'neighborhood': {'FeatureName

In [33]:
account_id = boto3.client("sts").get_caller_identity()["Account"]
print(account_id)

baseline_feature_group_resolved_output_s3_uri = (
    baseline_feature_group.describe()
    .get("OfflineStoreConfig")
    .get("S3StorageConfig")
    .get("ResolvedOutputS3Uri")
)

baseline_feature_group_s3_prefix = baseline_feature_group_resolved_output_s3_uri.replace(
    f"s3://housingdatafiles/feature-store4/002969758287", ""
)

002969758287


In [34]:
baseline_feature_group_resolved_output_s3_uri

's3://housingdatafiles/feature-store4/002969758287/sagemaker/us-east-1/offline-store/baseline_feature_group-1730172276/data'

In [ ]:
s3_client = boto3.client("s3", region_name=region)

offline_store_contents = None
while offline_store_contents is None:
    objects_in_bucket = s3_client.list_objects(
        Bucket=bucket_name, Prefix=baseline_feature_group_s3_prefix
    )
    if "Contents" in objects_in_bucket and len(objects_in_bucket["Contents"]) > 1:
        offline_store_contents = objects_in_bucket["Contents"]
    else:
        print("Waiting for data in offline store...\n")
        sleep(60)

print("Data available.")

In [38]:
import boto3

# Initialize Feature Store runtime client
featurestore_runtime = boto3.client("sagemaker-featurestore-runtime", region_name=region)

# Function to query a specific neighborhood
def query_neighborhood(feature_group_name, neighborhood_name):
    response = featurestore_runtime.get_record(
        FeatureGroupName=feature_group_name,
        RecordIdentifierValueAsString=neighborhood_name
    )
    return response

# Define the feature group name
feature_group_name = "baseline_feature_group"

# List of neighborhoods to query
neighborhoods = ["Brooktree", "Fisherman's Wharf", "Los Osos"]

# Query each neighborhood and print the results
for neighborhood in neighborhoods:
    result = query_neighborhood(feature_group_name, neighborhood)
    print(f"Feature values for {neighborhood}:")
    print(result)
    print("\n")


Feature values for Brooktree:
{'ResponseMetadata': {'RequestId': '2dc40edc-174a-487d-a1f4-aa7d8432be66', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2dc40edc-174a-487d-a1f4-aa7d8432be66', 'content-type': 'application/json', 'content-length': '1552', 'date': 'Tue, 29 Oct 2024 03:43:45 GMT'}, 'RetryAttempts': 0}, 'Record': [{'FeatureName': 'longitude', 'ValueAsString': '-121.88'}, {'FeatureName': 'latitude', 'ValueAsString': '37.4'}, {'FeatureName': 'housing_median_age', 'ValueAsString': '9.0'}, {'FeatureName': 'total_rooms', 'ValueAsString': '6751.0'}, {'FeatureName': 'population', 'ValueAsString': '4240.0'}, {'FeatureName': 'households', 'ValueAsString': '1438.0'}, {'FeatureName': 'median_income', 'ValueAsString': '5.34'}, {'FeatureName': 'median_house_value', 'ValueAsString': '257400.0'}, {'FeatureName': 'ocean_proximity', 'ValueAsString': '<1H OCEAN'}, {'FeatureName': 'neighborhood-political', 'ValueAsString': 'Brooktree'}, {'FeatureName': 'neighborhood', 'ValueAsStri



Here are the feature values retrieved from the Feature Store for each neighborhood:

1. Brooktree
Location:
Longitude: -121.88
Latitude: 37.4
Housing Details:
Housing Median Age: 9.0
Total Rooms: 6751.0
Population: 4240.0
Households: 1438.0
Median Income: 5.34
Median House Value: 257,400.0
Ocean Proximity: <1H OCEAN
Ocean Proximity Encodings:
<1H OCEAN: 1
Inland, Island, Near Bay, Near Ocean: 0
Derived Features:
Median House Age Grouped: 0.0
Event Time: 1730172276.0

2. Fisherman’s Wharf
Location:
Longitude: -122.42
Latitude: 37.81
Housing Details:
Housing Median Age: 52.0
Total Rooms: 1314.0
Total Bedrooms: 317.0
Population: 473.0
Households: 250.0
Median Income: 4.3472
Median House Value: 500,000.0 (capped)
Ocean Proximity: NEAR BAY
Ocean Proximity Encodings:
Near Bay: 1
Other categories: 0
Derived Features:
Median House Age Grouped: 50.0
Bedrooms per Household: 1.268
Event Time: 1730172276.0

3. Los Osos
Location:
Longitude: -120.67
Latitude: 35.3
Housing Details:
Housing Median Age: 32.0
Total Rooms: 4202.0
Total Bedrooms: 986.0
Population: 2309.0
Households: 956.0
Median Income: 2.2165
Median House Value: 231,700.0
Ocean Proximity: NEAR OCEAN
Ocean Proximity Encodings:
Near Ocean: 1
Other categories: 0
Derived Features:
Median House Age Grouped: 30.0
Bedrooms per Household: 1.031
Event Time: 1730172276.0